In [1]:

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

BASE_DIR = os.getcwd()
ENV_PATH = os.path.join("C:\\Users\diego\PycharmProjects\HultieChatbot", ".env")

load_dotenv(ENV_PATH)
openai_api_key = os.getenv("OPENAI_API_KEY")

def print_colored(text, color_code):
    print(f"\033[{color_code}m{text}\033[0m")

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path="Bases-CAP-Innovacion-2024_-VF.pdf"

loader= PyPDFLoader(file_path)
docs=loader.load()


text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=800, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs)

embd = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embd,
)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
) 

In [3]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class RouteQuery(BaseModel):
    datasource: Literal["general", "bases_del_concurso","fechas_del_concurso"] = Field(
        ...,
        description="Dada una pregunta, clasificala como general,bases del concurso, fechas del concurso.",
    )
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

In [6]:
system = """Tomas el rol de un agente clasificador profesional, que a partir de la pregunta del usuario
la clasifica como: general, bases del concurso o fechas del concurso.Clasificala como bases del concurso, cuando
la pregunta del usuario tenga algo relacionado a las reglas,objetivo o vision del concurso, las fechas cuando te pregunte acerca
del cronograma o cuando se va a llevar algo. Si algo no cumple con estas dos especificaciones, clasificalo como general
"""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
question_router = route_prompt | structured_llm_router
print_colored(question_router.invoke({"question": "Dame el cronograma"}),32)

datasource='fechas_del_concurso'


In [7]:
class GradeDocuments(BaseModel):
    binary_score: str = Field(
        description="Los documentos son relevantes para la pregunta del usuario, 'si' o 'no'"
    )
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

In [9]:
system = """
        Eres un evaluador encargado de asignar si un documento extraido de la vectorstore es relevante para responder la pregunta del usuario
        o no.
        Si el documento contiene keyword(s) o significado semantico relacionado a la pregunta del usuario, calificalo como relevante. Esto 
        no debe ser un test stricto, el objetivo es simplemente filtrar retrievals erroneos. Da una puntuación binaria de 'si' o 'no' para
        indicar si un documento es o no relevante para la pregunta del usuario"""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "Taller general de procedimientos para ganadores"
docs = retriever.invoke(question)
doc_txt = docs[0].page_content
print(doc_txt)
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

11 
 
 
10.   Otras consideraciones 
 
La Universidad se reserva el derecho de revocar la postulación o la ejecución del proyecto, según 
corresponda, cuando haya culminado el vínculo laboral o c ontractual del docente líder de l 
proyecto. De igual manera, en los casos en los que el docente líd er del proyecto registre una 
sanción como resultado de un procedimiento disciplinario contra él, en los últimos dos años, a 
través de las instancias competentes de la Universidad.  
 
Las circunstancias no descritas en las presentes bases serán resueltas por  el vicerrector de 
investigación y el director de la DFI con la asesoría que consideren pertinentes. 
 
11.  Cronograma de postulación 
 
 
El cronograma de postulación es el siguiente: 
 
 Inicio Fin 
Publicación de las bases 07/03/2024 
Registro de propuestas 01/04/2024 29/04/2024 
Publicación de resultados 31/07/2024 
Taller general de procedimientos para ganadores 06/08/2024 
Planificación operativa 
 
08/08/2024 09/09/2024 
Generaci

In [10]:
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Eres un asistente que responde dudas. Apartir de información sacada del vector store, responde la pregunta del usuario. Si no sabes la respuesta,
            di explicitamente que no tienes esa información. Se conciso en tus respuesta, que no pasen las 4 oraciones.
            Pregunta del usuario: {question}
            Contexto (Información sacada del vector store):{context}
            Respuesta:
            """
        ),
    ]
)


llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = prompt | llm | StrOutputParser()


generation = rag_chain.invoke({"context": docs, "question": question})
print_colored(generation,33)

El Taller General de Procedimientos para Ganadores está programado para el 6 de agosto de 2024. Este taller es parte del cronograma de postulación relacionado con los proyectos ganadores. Si necesitas más información sobre el taller o el proceso, puedes comunicarte con el Área de Promoción y Selección de la Dirección de Fomento de la Investigación.


In [11]:

class GradeHallucinations(BaseModel):
    binary_score: str = Field(
        description="La respuesta se basa en las fuentes: si o no"
    )

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)
system = """Eres un evaluador que clasifica si una generación de un llm esta basada en la información veridica entregada de las fuentes.
            Tu clasificación es binaria, debes clasificarlo como 'si', si es que si esta basada en la información entregada o 'no' si es que
            se esta inventando información o no sigue la información de las fuentes. Si la información veridica entregada dice: 'No hay información' significa que no se ha encontrado información verídica relevante, asi que en ese caso, clasificalo como 'si', si la respuesta del agente refleja desconocimiento. Por otro lado, clasificalo como 'no' si es que se inventa una respuesta."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Información entregada por las fuentes: \n\n {documents} \n\n Generación de LLM: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='si')

In [12]:
class GradeAnswer(BaseModel):
    binary_score: str = Field(
        description="La respuesta esta relacionada a la pregunta: si o no"
    )



llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)


system = """ Eres un evaluador que clasifica si la respuesta que genero un llm esta relacionada a la pregunta del usuario o si la resuelve.
            Clasifica de manera binaria con 'si' o 'no'. Donde 'si' significa que la pregunta del usuario ha sido resuelta o esta relacionada."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='si')

In [13]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
system = """Eres un re-escritor de preguntas del usuario, convirtiendolas en una mejor versión optimizada para 'vectorstore retrieval'.
            Analiza la pregunta y trata de razonar acerca de intento semantico de la pregunta. Siempre en español."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Pregunta inicial: \n\n {question} \n Formula una mejor version.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'Versión optimizada: \n\n"¿Cuáles son los procedimientos generales que se enseñan en el taller para los ganadores?"'

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [14]:
#Grafoo
from typing import List
from langgraph.graph import END, StateGraph, START
from typing_extensions import TypedDict


class GraphState(TypedDict):
    question: str #Pregunta del usuariooo
    generation: str #Respuesta del modelo
    documents: List[str] #Lista de documentos
    tries_retrieving: int 

In [15]:
from langchain.schema import Document

def init(state):
    return {"tries_retrieving": 0}


def retrieve(state):
    print_colored("---RETRIEVE---",33)
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    print_colored("---LLM:GENERACION---",32)
    question = state["question"]
    documents = state["documents"]
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    
    question = state["question"]
    documents = state["documents"]
    tries = state["tries_retrieving"]
    print_colored(f"---VERIFICANDO RELAVANCIA DE INFORMACIÓN---INTENTO:{tries}",35)
    filtered_docs = []
    if tries < 3:
        for d in documents:
            score = retrieval_grader.invoke(
                {"question": question, "document": d.page_content}
            )
            grade = score.binary_score
            if grade == "si":
                print_colored("---DOCUMENTO RELEVANTE---",34)
                filtered_docs.append(d)
            else:
                print("---DOCUMENT NO RELEVANTE---",31)
                continue
    else:
        filtered_docs.append("Sin informacion")
    return {"tries_retrieving":tries+1,"documents": filtered_docs, "question": question}


def transform_query(state):
    
    print_colored("---TRANSFORMAR PREGUNTA A UNA VERSION OPTIMIZADA PARA VECTORSTORE---",32)
    question = state["question"]
    documents = state["documents"]
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def general(state):
    print_colored("---LLM:GENERACION---",32)
    question = state["question"]
    generation = rag_chain.invoke({"context": "Información general, responder la pregunta de manera conscisa y si no tiene nada que ver con el concurso,redirigirlo", "question": question})
    return {"question": question, "generation": generation}


def route_question(state):
    print_colored("---CLASIFICACIÓN DE PREGUNTA---",36)
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "bases_del_concurso":
        print_colored("---PREGUNTA CLASIFICADA COMO: bases_del_concurso---",36)
        return "vectorstore"
    elif source.datasource == "fechas_del_concurso":
        print_colored("---PREGUNTA CLASIFICADA COMO: fechas_del_concurso---",36)
        return "vectorstore"
    elif source.datasource == "general":
        print_colored("---PREGUNTA CLASIFICADA COMO: general---",36)
        return "general"


def decide_to_generate(state):

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        print_colored(
            "---NINGÚN DOCUMENTO ES RELEVANTE PARA RESPONDER LA PREGUNTA---",31
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print_colored("---HAY DOCUMENTOS RELEVANTES, GENERANDO...---",32)
        return "generate"


def grade_generation_v_documents_and_question(state):
    print_colored("---VERIFICANDO ALUCINACIONES---",33)
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    
    print_colored(f"generacion:{generation}-",33)
    print_colored(f"documentos:{documents}",33)
    grade = score.binary_score
    if grade == "si":
        print_colored("---NO HAY ALUCINACIONES---",33)
        return "useful"
    else:
        print_colored("---EXISTEN ALUCIONACIONES, REINTENTANDO---",31)
        return "not supported"

In [16]:
workflow = StateGraph(GraphState)
workflow.add_node("init",init)
workflow.add_node("general", general)  
workflow.add_node("retrieve", retrieve)  
workflow.add_node("grade_documents", grade_documents) 
workflow.add_node("generate", generate)  
workflow.add_node("transform_query", transform_query)  
workflow.add_edge(START,"init")
workflow.add_conditional_edges(
    "init",
    route_question,
    {
        "general": "general",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("general",END)
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
    },
)
app = workflow.compile()

In [20]:
from pprint import pprint

inputs = {
    "question": "Que visión tiene el concurso?"
}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Node '{key}':")
    pprint("\n---\n")
pprint(value["generation"])

---CLASIFICACIÓN DE PREGUNTA---
---PREGUNTA CLASIFICADA COMO: bases_del_concurso---
"Node 'init':"
'\n---\n'
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---VERIFICANDO RELAVANCIA DE INFORMACIÓN---INTENTO:0
---DOCUMENT NO RELEVANTE--- 31
---DOCUMENTO RELEVANTE---
---DOCUMENT NO RELEVANTE--- 31
---ASSESS GRADED DOCUMENTS---
---HAY DOCUMENTOS RELEVANTES, GENERANDO...---
"Node 'grade_documents':"
'\n---\n'
---LLM:GENERACION---
---VERIFICANDO ALUCINACIONES---
generacion:La visión del concurso es dinamizar el ecosistema de innovación de la PUCP, promoviendo la generación de iniciativas de innovación y maduración tecnológica. Se busca fomentar una cultura de innovación y desarrollar capacidades que amplíen la comunidad de innovadores y emprendedores en la universidad. Además, se pretende que los proyectos tengan un impacto positivo en la sociedad y contribuyan al desarrollo interdisciplinario.-
documentos:[Document(metadata={'page': 0, 'source': 'Bases-CAP-Innovacion-2024_-VF.pdf'}, page_cont